# Conversational Chain

This program is intended to demo the use of the following: (Chương trình này nhằm mục đích giới thiệu việc sử dụng các tính năng sau:)
- Extracts the contects of a webpage, chunks and loads the chunks (documents) into a FAISS db <br>
<i>→(Trích xuất nội dung của trang web, chia thành các phần và tải các phần (tài liệu) vào db FAISS.)</i>
- Creates a sample conversaion, uses a "create_history_aware_retriever" retrieval chain <br>
<i>→(Tạo một cuộc trò chuyện mẫu bằng cách sử dụng chuỗi truy xuất "create_history_aware_retriever".)</i>
- The last step is to pass the history and ask a follow up question <br>
<i>→(Bước cuối cùng là truyền lại lịch sử và đặt câu hỏi tiếp theo.)</i>

In [2]:
import os
from dotenv import load_dotenv
import openai
from langchain_openai import ChatOpenAI

In [9]:
# Setup model
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
openai.api_key = api_key
llm = ChatOpenAI(temperature=0)

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_retrieval_chain, create_history_aware_retriever
from langchain_core.messages import HumanMessage, AIMessage

In [8]:
# Load data from website
loader = WebBaseLoader("https://www.wikihow.com/Do-Yoga")
docs = loader.load()
print(len(docs))
docs

1


[Document(page_content='\n\n\n\nHow to Do Yoga: Tips & Poses for Beginners\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to ContentQuizzesPRO\n\n\t\tCourses\n\n\t\n\n\t\tGuides\n\n\t\tNew\n\n\n\t\tTech Help Pro\n\n\t\n\n\t\tExpert Videos\n\n\t\n\n\t\tAbout wikiHow Pro\n\n\t\n\n\t\tUpgrade\n\n\t\n\n\t\tSign In\n\n\t\n\nQUIZZESEDIT\n\n\t\tEdit this Article\n\n\t\n\nEXPLORE\n\nTech Help ProAbout UsRandom ArticleQuizzes\n\n\nRequest a New ArticleCommunity DashboardThis Or That Game\n\n\nPopular Categories\n\n\nArts and EntertainmentArtworkBooksMoviesComputers and ElectronicsComputersPhone SkillsTechnology HacksHealthMen\'s HealthMental HealthWomen\'s HealthRelationshipsDatingLoveRelationship Issues\n\n\nHobbies and CraftsCraftsDrawingGamesEducation & CommunicationCommunication SkillsPersonal DevelopmentStudyingPersonal Care and StyleFashionHair CarePersonal HygieneYouthPersonal CareSchool StuffDat

In [10]:
# Split into chunks
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
print(len(documents))
for document in documents:
    print("-------------------------------")
    print(document)

14
-------------------------------
page_content="How to Do Yoga: Tips & Poses for Beginners\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to ContentQuizzesPRO\n\n\t\tCourses\n\n\t\n\n\t\tGuides\n\n\t\tNew\n\n\n\t\tTech Help Pro\n\n\t\n\n\t\tExpert Videos\n\n\t\n\n\t\tAbout wikiHow Pro\n\n\t\n\n\t\tUpgrade\n\n\t\n\n\t\tSign In\n\n\t\n\nQUIZZESEDIT\n\n\t\tEdit this Article\n\n\t\n\nEXPLORE\n\nTech Help ProAbout UsRandom ArticleQuizzes\n\n\nRequest a New ArticleCommunity DashboardThis Or That Game\n\n\nPopular Categories\n\n\nArts and EntertainmentArtworkBooksMoviesComputers and ElectronicsComputersPhone SkillsTechnology HacksHealthMen's HealthMental HealthWomen's HealthRelationshipsDatingLoveRelationship Issues\n\n\nHobbies and CraftsCraftsDrawingGamesEducation & CommunicationCommunication SkillsPersonal DevelopmentStudyingPersonal Care and StyleFashionHair CarePersonal HygieneYouthPersonal Care

In [16]:
# store chunks to FAISS
embeddings = OpenAIEmbeddings()
vector = FAISS.from_documents(documents,embeddings) # ingest documents into the vector store (nhập tài liệu vào vector store)
print(vector.index.ntotal) 
print(vector.index_to_docstore_id)

14
{0: '01aff58f-dd0a-4cba-8318-ec3cec7e17ff', 1: '8f45aa23-24b9-4e7f-ad40-86c8dbd6aa8b', 2: 'bf87d97a-6f37-41d6-8e6d-766d75b41be3', 3: '035de03a-78b1-4818-a5dc-9e41f7de2f75', 4: '8f34e2b3-91c7-416b-b58b-18a6510ac9a4', 5: '767d1eb9-7d06-4eb2-a1a1-e3290c621b75', 6: '5e991245-a4de-4fb5-8a6e-f76670b99c55', 7: '3a8e2503-dcfb-4f50-912c-4df615ee3b08', 8: '9e28eb28-738f-461f-ae11-f11d9fa3c3be', 9: '1b079b24-1344-451e-b07b-44cb86750d2a', 10: 'aeed65d8-f52e-43b5-95a6-e4054039587d', 11: 'bc2c5ec6-906a-4c23-ab38-016651b51547', 12: '8de3567f-0af6-49f0-8d49-93b253670e64', 13: '9805879d-1dc1-4312-847a-e1758654c459'}
